<a href="https://colab.research.google.com/github/saktiworkstation/road-to-ai-developer/blob/main/Kode_UTS_BigDataLanjut.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [53]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, avg, corr
import pandas as pd

In [4]:
# Inisialisasi SparkSession
spark = SparkSession.builder.appName("SaktiUTSBigDataLanjut").getOrCreate()

In [5]:
# Membaca dataset dari CSV
df = spark.read.csv("/content/Spotify_Youtube.csv", header=True, inferSchema=True)

In [6]:
# Menampilkan lima baris pertama dari dataset
df.show(5)

+--------------------+--------+--------------------+--------------+----------+----------+--------------------+------------+------+----+--------+-----------+------------+----------------+--------+-------+-------+-----------+--------------------+--------------------+--------+-----------+---------+--------+--------------------+--------+--------------+------+
|                 _c0|  Artist|         Url_spotify|         Track|     Album|Album_type|                 Uri|Danceability|Energy| Key|Loudness|Speechiness|Acousticness|Instrumentalness|Liveness|Valence|  Tempo|Duration_ms|         Url_youtube|               Title| Channel|      Views|    Likes|Comments|         Description|Licensed|official_video|Stream|
+--------------------+--------+--------------------+--------------+----------+----------+--------------------+------------+------+----+--------+-----------+------------+----------------+--------+-------+-------+-----------+--------------------+--------------------+--------+----------

In [7]:
# Memeriksa tipe data dari masing-masing kolom
df.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- Artist: string (nullable = true)
 |-- Url_spotify: string (nullable = true)
 |-- Track: string (nullable = true)
 |-- Album: string (nullable = true)
 |-- Album_type: string (nullable = true)
 |-- Uri: string (nullable = true)
 |-- Danceability: string (nullable = true)
 |-- Energy: string (nullable = true)
 |-- Key: string (nullable = true)
 |-- Loudness: string (nullable = true)
 |-- Speechiness: string (nullable = true)
 |-- Acousticness: string (nullable = true)
 |-- Instrumentalness: string (nullable = true)
 |-- Liveness: string (nullable = true)
 |-- Valence: string (nullable = true)
 |-- Tempo: string (nullable = true)
 |-- Duration_ms: string (nullable = true)
 |-- Url_youtube: string (nullable = true)
 |-- Title: string (nullable = true)
 |-- Channel: string (nullable = true)
 |-- Views: string (nullable = true)
 |-- Likes: string (nullable = true)
 |-- Comments: string (nullable = true)
 |-- Description: string (nullable = true)
 

In [14]:
# Menghitung jumlah nilai null per kolom
df.select([col(c).isNull().alias(c) for c in df.columns]).show()

+-----+------+-----------+-----+-----+----------+-----+------------+------+-----+--------+-----------+------------+----------------+--------+-------+-----+-----------+-----------+-----+-------+-----+-----+--------+-----------+--------+--------------+------+
|  _c0|Artist|Url_spotify|Track|Album|Album_type|  Uri|Danceability|Energy|  Key|Loudness|Speechiness|Acousticness|Instrumentalness|Liveness|Valence|Tempo|Duration_ms|Url_youtube|Title|Channel|Views|Likes|Comments|Description|Licensed|official_video|Stream|
+-----+------+-----------+-----+-----+----------+-----+------------+------+-----+--------+-----------+------------+----------------+--------+-------+-----+-----------+-----------+-----+-------+-----+-----+--------+-----------+--------+--------------+------+
|false| false|      false|false|false|     false|false|       false| false|false|   false|      false|       false|           false|   false|  false|false|      false|      false|false|  false|false|false|   false|      false|

In [9]:
# Mendapatkan deskripsi secara umum (statistik dasar)
df.describe().show()

+-------+--------------------+------+--------------------+----------------------+----------------------------------+--------------------+--------------------+--------------------+--------------------+-------------------+--------------------+--------------------+--------------------+--------------------+-----------------+--------------------+------------------+--------------------+------------------+--------------------+--------------------+--------------------+-------------------+------------------+--------------------+--------------------+--------------------+--------------------+
|summary|                 _c0|Artist|         Url_spotify|                 Track|                             Album|          Album_type|                 Uri|        Danceability|              Energy|                Key|            Loudness|         Speechiness|        Acousticness|    Instrumentalness|         Liveness|             Valence|             Tempo|         Duration_ms|       Url_youtube|         

In [10]:
# Ringkasan statistik dari 'Danceability', 'Energy', 'Valence'
df.select('danceability', 'energy', 'valence').describe().show()

+-------+--------------------+--------------------+--------------------+
|summary|        danceability|              energy|             valence|
+-------+--------------------+--------------------+--------------------+
|  count|               22881|               22385|               21152|
|   mean|    554473.105467201|  358093.43691556354|  44533.389922250106|
| stddev|2.6332502472555384E7|1.2778964568124782E7|  2326257.7180884588|
|    min|                    |      The Bitter End|           #popindie|
|    max|” “Take Me In You...|” “Writer In The ...|” “The Princess &...|
+-------+--------------------+--------------------+--------------------+



In [11]:
# Identifikasi 5 lagu dengan 'Speechiness' tertinggi
speechiest_songs = df.orderBy(col("speechiness").desc()).limit(5)
speechiest_songs.show()

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----------------+--------------------+------------+--------------------+--------------------+
|                 _c0|              Artist|         Url_spotify|               Track|               Album|          Album_type|                 Uri|        Danceability|              Energy|                 Key|            Loudness|         Speechiness|        Acousticness|    Instrumentalness|            Liveness|             Valence|               Tempo|         Duration_ms|         Url_youtube|               Title|

In [12]:
# Menghitung rata-rata dari atribut 'Tempo'
tempo_avg = df.select(avg("tempo")).collect()[0][0]
print(f"Rata-rata Tempo: {tempo_avg}")

Rata-rata Tempo: 3047.6026549845583


In [15]:
# Membandingkan rata-rata durasi lagu dengan tempo di atas dan di bawah 120 BPM
duration_above_120 = df.filter(col("tempo") > 120).select(avg("duration_ms")).collect()[0][0]
duration_below_120 = df.filter(col("tempo") <= 120).select(avg("duration_ms")).collect()[0][0]
print(f"Rata-rata durasi (Tempo > 120 BPM): {duration_above_120}")
print(f"Rata-rata durasi (Tempo <= 120 BPM): {duration_below_120}")

Rata-rata durasi (Tempo > 120 BPM): 221268.44684098076
Rata-rata durasi (Tempo <= 120 BPM): 227419.06441330028


In [18]:
# Temukan lagu dengan tingkat speechiness di atas 0.5 dan korelasi dengan jumlah komentar di YouTube
high_speechiness_songs = df.filter(col("speechiness") > 0.5)
high_speechiness_songs.select("speechiness", "comments").show()

+-----------+--------+
|speechiness|comments|
+-----------+--------+
|       2017|    NULL|
|      0.591|   120.0|
|      0.522|   398.0|
|      0.858|     5.0|
|      0.836|     5.0|
|      0.899|     5.0|
|      0.761|     5.0|
|      0.865|     5.0|
|      0.821|     5.0|
|      0.909|     5.0|
|      0.706|     5.0|
|  249630617|    NULL|
|  186034963|    NULL|
|  147480448|    NULL|
|   11850761|    NULL|
|    5502218|    NULL|
|      0.922|    NULL|
|      0.939|    NULL|
|      0.947|    NULL|
|      0.949|    NULL|
+-----------+--------+
only showing top 20 rows



In [54]:
# Menghitung korelasi antara speechiness dan jumlah komentar
correlation = df.filter(col("speechiness") > 0.5).select(corr("speechiness", "comments")).collect()[0][0]
print(f"Korelasi antara speechiness dan jumlah komentar: {correlation}")

Korelasi antara speechiness dan jumlah komentar: -0.2545760870285546


In [19]:
# RDD Operations
rdd = df.rdd

In [46]:
# 1. Menggunakan reduceByKey untuk mendapatkan jumlah komentar
comment_rdd = rdd.filter(lambda row: row.asDict().get('Track') is not None or row.asDict().get('Track') is not None)
comment_rdd = comment_rdd.map(lambda row: (row.asDict().get('Track', row.asDict().get('Track')), row.asDict().get('Comments', 0)))
comment_rdd = comment_rdd.reduceByKey(lambda x, y: (x or '') + (y or '')) # Handle None values by replacing them with 0
comment_rdd.collect()

[('New Gold (feat. Tame Impala and Bootie Brown)', '7399.07399.07399.0'),
 ('323850327', None),
 ('New Gold (feat. Tame Impala and Bootie Brown) - Dom Dolla Remix',
  '241.0241.0'),
 ("She's My Collar (feat. Kali Uchis)", '260.0'),
 ('159605929', None),
 ('Cracker Island (feat. Thundercat)', '20296.020295.0'),
 (' Murdoc Niccals', ''),
 (' Stars Redmond', ''),
 (' Josh Berman"', ''),
 ('191074713', None),
 ('Californication', '121452.0'),
 (' and #16 on the UK charts', None),
 (' be my very own constellation', None),
 ('1055738398', None),
 ('Under the Bridge', '32761.0'),
 (' you better come to ', None),
 ('866464951', None),
 (' and won six Grammy Awards', None),
 (' yeah', ''),
 ('550067391', None),
 ('Give It Away', '16029.03054.0'),
 ('301947159', None),
 ('Dark Necessities', '56516.0'),
 ('1041736808', None),
 ('P.I.M.P.', '60552.0'),
 ('541728561', None),
 ('332175400', None),
 ('Disco Inferno', '12038.0'),
 ('202477187', None),
 ('21 Questions', '54805.0'),
 ("If I Can't", '14.

In [50]:
# 2. Menggunakan sortByKey untuk mengurutkan dataset berdasarkan nama lagu
sorted_rdd = rdd.map(lambda row: (row.asDict().get('Track') or row.asDict().get('Track'), row)).filter(lambda x: x[0] is not None).sortByKey()
sorted_rdd.take(5)

[(' ',
  Row(_c0='She might as well be attached to me', Artist=' now she can’t go a day without chat to me', Url_spotify=' said she love the way mi give her love naturally', Track=' ', Album=None, Album_type=None, Uri=None, Danceability=None, Energy=None, Key=None, Loudness=None, Speechiness=None, Acousticness=None, Instrumentalness=None, Liveness=None, Valence=None, Tempo=None, Duration_ms=None, Url_youtube=None, Title=None, Channel=None, Views=None, Likes=None, Comments=None, Description=None, Licensed=None, official_video=None, Stream=None)),
 (' ',
  Row(_c0='And she can’t stay away', Artist=' snap back to me', Url_spotify=' she gridlock to me', Track=' ', Album=None, Album_type=None, Uri=None, Danceability=None, Energy=None, Key=None, Loudness=None, Speechiness=None, Acousticness=None, Instrumentalness=None, Liveness=None, Valence=None, Tempo=None, Duration_ms=None, Url_youtube=None, Title=None, Channel=None, Views=None, Likes=None, Comments=None, Description=None, Licensed=None, 

In [51]:
# 3. Menggunakan flatMap untuk memisahkan track_name menjadi list kata
title_words_rdd = rdd.flatMap(lambda row: (row.asDict().get('Track') or row.asDict().get('Track') or "").split(" "))
title_words_count = title_words_rdd.map(lambda word: (word, 1)).reduceByKey(lambda x, y: x + y)
title_words_count.collect()

[('Feel', 49),
 ('Good', 100),
 ('', 500047),
 ('New', 75),
 ('Tame', 10),
 ('Impala', 10),
 ('Melancholy', 1),
 ('Hill', 10),
 ('Eastwood', 1),
 ('Rock', 68),
 ('The', 1291),
 ('House', 33),
 ('Comes', 19),
 ('acclaimed', 22),
 ('album', 96),
 ('was', 125),
 ('in', 643),
 ('BRIT', 1),
 ('Grammy', 61),
 ('band’s', 7),
 ('subsequent', 1),
 ('are', 56),
 ('Days', 44),
 ('(2005)', 1),
 ('323850327', 1),
 ('Dom', 5),
 ('Kali', 9),
 ('159605929', 1),
 ('Island', 6),
 ('Thundercat)', 2),
 ('Fatih', 2),
 ('Dogan', 2),
 ('Steph', 3),
 ('Anjo', 3),
 ('Rudolf', 2),
 ('Stars', 20),
 ('Redmond', 2),
 ('Josh', 11),
 ('191074713', 1),
 ('Californication', 1),
 ('#16', 1),
 ('“Dani', 9),
 ('very', 8),
 ('own', 39),
 ('1055738398', 1),
 ('Bridge', 7),
 ('no', 166),
 ("Can't", 72),
 ('Stop', 41),
 ('866464951', 1),
 ('Scar', 1),
 ('Tissue', 1),
 ('Snow', 10),
 ('(Hey', 1),
 ('Oh)', 3),
 ('Dani', 3),
 ('550067391', 1),
 ('Give', 44),
 ('It', 391),
 ('Away', 60),
 ('301947159', 1),
 ('Da', 44),
 ('Club',